In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.9 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 33.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595742 sha256=10820ad35d0da6722215e8637ce8aff8921956de7278e0c8f8c0bdacc46adc50
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 27.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-f3jvhh41
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-f3jvhh41
     |████████████████████████████████| 132 kB 34.2 MB/s 
     |████████████████████████████████| 4.5 MB 53.5 MB/s 
     |████████████████████████████████| 4.7 MB 59.0 MB/s 
     |████████████████████████████████| 596 kB 69.7 MB/s 
     |████████████████████████████████| 6.6 MB 49.2 MB/s 
     |████████████████████████████████| 101 kB 14.9 MB/s 
     |████████████████████████████████| 9.0 MB 57.2 MB/s 
     |████████████████████████████████| 79 kB 9.7 MB/s 
     |████████████████████████████████| 127 kB 79.2 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=699383a77cabcc81356f08adbcb6b8763a6d036f8b9bb4bc34fe4a5e328fdb3b
  Stored in director

In [14]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
# 이후에 데이터 및 파일들을 저장할 경로를 path로 통일하여 따로 저장

path = '/content/gdrive/MyDrive/Colab Notebooks/CUAI 하계프로젝트-NLP/초기 모델링 진행/data/'
path = '/content/drive/MyDrive/Colab Notebooks/CUAI 하계프로젝트-NLP/초기 모델링 진행/data/'

In [10]:
#GPU 사용
device = torch.device("cuda:0")

- KoBERT 모델에 파라미터를 변경하여 학습을 진행했습니다
- 사용한 데이터는 데이콘에서 제공된 데이터를 사용했습니다
- Private 점수 가장 높게 나온 파라미터입니다.


In [11]:
# final_data 로드

data = pd.read_excel(path + 'final_data.xlsx', engine='openpyxl')

In [12]:
data.tail(3)

,Unnamed: 0,document,label
303472,303472,당장 내일 밥은 어떡하고 내 병원비는 어쩌지,0
303473,303473,응 이제 다들 독립해가지고 명절 아니면 찾아오질 않아,0
303474,303474,안부전화라도 해주면 좋을 텐데,0


In [15]:
# 책에 train과 test 데이터를 나눴으므로 train_test_split을 이용해 final_data를 분리
# test_size = 0.25가 defalut, 랜덤 시드 42로 고정
X_train, X_test, y_train, y_test = train_test_split(data['document'],data['label'], 
                                                    test_size = 0.25, stratify = data['label'], random_state=42)

In [16]:
# 다시 train과 test끼리 묶어주고 이것을 일단 txt 파일로 저장
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data.to_csv(path + 'final_train_data.txt', sep = '\t', index = False)
test_data.to_csv(path + 'final_test_data.txt', sep = '\t', index = False)

In [17]:
train_data.head(3)

,document,label
196891,혼자 앉아있는데 너무 민망해,0
21067,다시 보고 싶다 앙코르 해주면 좋겠다,1
291642,야근을 많이 하는데 돌아오는 게 없네,0


In [18]:
train_dataset = []
for sen, label in zip(train_data['document'], train_data['label']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(label))

  train_dataset.append(data_train)

In [19]:
train_dataset

[['혼자 앉아있는데 너무 민망해', '0'],
 ['다시 보고 싶다 앙코르 해주면 좋겠다', '1'],
 ['야근을 많이 하는데 돌아오는 게 없네', '0'],
 ['너무재밌당서은기느린대사가조금거슬림안변재희마루재희 강제키스장면이너무좋다안변재희 강제 키스 장면 있으면 좋을 거 같다고 생각했는데 딱나왔ㅋㅋ는한재희랑이어졌으면좋겠다박시연이더예쁘당박변은녹음하고있을때에신고하면되지아강마루너무좋다안변도넘멋있다',
  '1'],
 ['결혼을 꼭 해야 할까', '0'],
 ['술도 안 먹고 운동 많이 한다고 건강한 건 아닌가 봐', '1'],
 ['나를 괴롭히던 친구가 전학을 간대', '1'],
 ['삼류에만 줄곧 출연하는 램버트', '0'],
 ['토요명화에서나 해줄 만한 식사가 한 로켄틱 코미디 그나마 비디오로 보길 잘했지', '0'],
 ['요즘 학생들은 미친 거 같아', '0'],
 ['딱히 안 봤어도 아쉽지는 않았을', '0'],
 ['난 열심히 노력해서 정시로 수시로 갈 수 있는 대학보다 더 좋은 대학에 가고 싶어', '0'],
 ['다음엔 더 잘 하겠지', '1'],
 ['아들 장가도 보내야 하고 또 우리 부부는 아픈 곳도 많아서 돈 쓸 일이 많아', '0'],
 ['기억재생 편집 재미나는 테마 아닌 결말 영화나 제대로 편집하지', '0'],
 ['솔직히 굿 ㅋㅋ 넘버 시는 살아남길 바랐는데 번째 극장판 어서 나오길', '1'],
 ['잔인하고 아름답고', '1'],
 ['최근 들어 부모님의 사이가 나빠져서 너무 괴로워', '0'],
 ['재정이 넉넉해서 두 아이를 모두 유학을 보내게 됐어', '1'],
 ['마치 지옥에서 시간을 달리면서 한 소녀가 다가오는 듯한 느낌의 해괴한 고작', '0'],
 ['퇴직하고 쉬면 무조건 좋을 줄 알았는데 휴식이 길어지니 무기력해져서 더 힘든 것 같아', '0'],
 ['오랜만에 오감을 만족시키는 한 영화감독의 감각이', '1'],
 ['전형적 영국식 코미디 새콤달콤하다', '1'],
 ['현재 나에 대해서 생각할 시간을 주는 

In [20]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [21]:
# Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5

In [22]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [23]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_dataset, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


RuntimeError: ignored

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.4).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    # test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

In [ ]:
'''
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(sentence):
  data = [sentence, '0']
  dataset_another = [data]
  logits = 0
  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
  
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)

      valid_length= valid_length
      label = label.long().to(device)

      out = model(token_ids, valid_length, segment_ids)

      for i in out:
        logits = i
        logits = logits.detach().cpu().numpy()
        logits = np.argmax(logits)

  return logits
  '''

In [ ]:
'''
label_answer = []
step = 0
for i in tqdm(test_data['document']):
  label_answer.append(predict(i))
'''